<a href="https://colab.research.google.com/github/Saheed7/anomaly-agent-demo/blob/main/Anomaly_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Anomaly_Agent_Demo

In [ ]:
# 1. Install Dependencies
!pip install pandas scikit-learn shap anthropic openai deepseek-py python-dotenv


In [ ]:
# 2. Import Libraries
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler
import shap
import joblib
import json
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from deepseek import DeepSeek

In [ ]:
# 3. Load API Keys (Create .env file with your keys)
load_dotenv()
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
claude_client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
deepseek_client = DeepSeek(api_key=os.getenv("DEEPSEEK_API_KEY"))

In [ ]:
# 4. Load Dataset (Synthetic example - replace with Edge-IIoTset/CIC-IoT 2023)
data = {
    'packet_size': np.random.randint(50, 1500, 1000),
    'protocol_type': np.random.choice(['Modbus', 'MQTT', 'CoAP', 'HTTP'], 1000),
    'packet_rate': np.random.uniform(1, 500, 1000),
    'duration': np.random.uniform(0.1, 60, 1000),
    'label': np.random.choice(['Benign', 'DDoS-UDP', 'PortScan', 'PLC-Injection'], 1000,
                             p=[0.7, 0.1, 0.1, 0.1])
}
df = pd.DataFrame(data)

In [ ]:
# 5. Preprocessing
def preprocess_data(df):
    # Encode categorical features
    df = pd.get_dummies(df, columns=['protocol_type'])

    # Normalize numerical features
    scaler = MinMaxScaler()
    numerical_features = ['packet_size', 'packet_rate', 'duration']
    df[numerical_features] = scaler.fit_transform(df[numerical_features])

    # Separate features and labels
    X = df.drop('label', axis=1)
    y = df['label']
    return X, y, scaler

X, y, scaler = preprocess_data(df)

In [ ]:
# 6. Train Models
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100),
    'SVM': SVC(kernel='rbf', C=1.0, probability=True),
    'Naïve Bayes': GaussianNB()
}

for name, model in models.items():
    model.fit(X, y)

In [ ]:
# 7. SHAP Analysis
explainer = shap.KernelExplainer(models['Random Forest'].predict_proba, shap.sample(X, 100))
shap_values = explainer.shap_values(X)

In [ ]:
# 8. LLM Agent Functions
def call_llm(llm_name, system_prompt, user_prompt):
    """Call different LLM APIs with standardized prompt structure"""
    if llm_name == "DeepSeek R1":
        response = deepseek_client.chat(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=512
        )
        return response.choices[0].message.content

    elif llm_name == "GPT-4o":
        response = openai_client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=512
        )
        return response.choices[0].message.content

    elif llm_name == "GPT-4o":
        response = openai_client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=512
        )
        return response.choices[0].message.content

    elif llm_name == "Claude 3.5":
        response = claude_client.messages.create(
            model="claude-3-5-sonnet-20240620",
            max_tokens=512,
            system=system_prompt,
            messages=[{"role": "user", "content": user_prompt}]
        )
        return response.content[0].text

    elif llm_name == "GPT-4o-mini":
        response = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=512
        )
        return response.choices[0].message.content

In [ ]:
# 9. Anomaly Agent Inference Pipeline
def anomaly_agent_inference(features, llm_name="DeepSeek R1", sensitivity="balanced"):
    # Preprocess input
    input_df = pd.DataFrame([features])
    input_df = pd.get_dummies(input_df)
    input_df = input_df.reindex(columns=X.columns, fill_value=0)

    # Get predictions from all models
    predictions = {}
    for name, model in models.items():
        proba = model.predict_proba(input_df)[0]
        top_class_idx = np.argsort(proba)[-1]
        predictions[name] = {
            "prediction": model.classes_[top_class_idx],
            "confidence": float(proba[top_class_idx]),
            "details": {model.classes_[i]: float(proba[i]) for i in range(len(proba))}
        }
    # Get SHAP explanation
    shap_exp = explainer.shap_values(input_df)
    shap_values_simplified = {
        feature: float(value)
        for feature, value in zip(X.columns, shap_exp[0][0])
    }

    # Prepare LLM prompt
    system_prompt = f"""
    You are an IoT security analyst. Analyze these model predictions and SHAP values to:
    1. Determine final prediction (Benign/Malicious with attack type)
    2. Explain reasoning using SHAP values
    3. Apply {sensitivity} sensitivity mode
    Output ONLY JSON with keys: prediction, confidence, explanation
    """
    user_prompt = f"""
    ## Model Predictions:
    {json.dumps(predictions, indent=2)}

    ## SHAP Values (Feature Contributions):
    {json.dumps(shap_values_simplified, indent=2)}

    ## Traffic Features:
    {json.dumps(features, indent=2)}
    """
    # Call LLM for final decision
    llm_response = call_llm(llm_name, system_prompt, user_prompt)

    try:
        # Extract JSON from LLM response
        json_start = llm_response.find('{')
        json_end = llm_response.rfind('}') + 1
        final_decision = json.loads(llm_response[json_start:json_end])
        return final_decision
    except:
        return {"error": "LLM response parsing failed", "raw_response": llm_response}


In [ ]:
# 10. Demo Execution
sample_features = {
    'packet_size': 1200,
    'packet_rate': 85.3,
    'duration': 4.2,
    'protocol_type': 'Modbus'
}

print("### DeepSeek R1 Results ###")
print(anomaly_agent_inference(sample_features, "DeepSeek R1"))

print("\n### GPT-4o Results ###")
print(anomaly_agent_inference(sample_features, "GPT-4o"))

print("\n### Claude 3.5 Results ###")
print(anomaly_agent_inference(sample_features, "Claude 3.5"))

print("\n### GPT-4o-mini Results ###")
print(anomaly_agent_inference(sample_features, "GPT-4o-mini"))





